In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import plotly.express as px

In [3]:
data = pd.read_csv('2017.csv')
data.head()

C:\Users\xfuen\AppData\Local\Temp\ipykernel_23772\1704392842.py:1: DtypeWarning: Columns (3,71,72,81,135,136,164,179,180) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('2017.csv')


,contract_transaction_unique_key,contract_award_unique_key,award_id_piid,modification_number,transaction_number,parent_award_agency_id,parent_award_agency_name,parent_award_id_piid,parent_award_modification_number,federal_action_obligation,...,highly_compensated_officer_2_amount,highly_compensated_officer_3_name,highly_compensated_officer_3_amount,highly_compensated_officer_4_name,highly_compensated_officer_4_amount,highly_compensated_officer_5_name,highly_compensated_officer_5_amount,usaspending_permalink,initial_report_date,last_modified_date
0,6800_4732_EPG16H01231_5_GS35F0119Y_0,CONT_AWD_EPG16H01231_6800_GS35F0119Y_4732,EPG16H01231,5,0.0,4732.0,FEDERAL ACQUISITION SERVICE,GS35F0119Y,PS1139REREPSMALLBUSS,5559.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/CONT_AWD_EPG...,2017-09-30,2020-03-27
1,6800_-NONE-_EPD17022_3_-NONE-_0,CONT_AWD_EPD17022_6800_-NONE-_-NONE-,EPD17022,3,0.0,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/CONT_AWD_EPD...,2017-09-30,2020-03-27
2,6800_6800_0008_0_EPW17022_0,CONT_AWD_0008_6800_EPW17022_6800,0008,0,0.0,6800.0,ENVIRONMENTAL PROTECTION AGENCY,EPW17022,0,50000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/CONT_AWD_000...,2017-09-30,2017-11-26
3,6800_-NONE-_EP159000050_3_-NONE-_0,CONT_AWD_EP159000050_6800_-NONE-_-NONE-,EP159000050,3,0.0,NaN,NaN,NaN,NaN,1813.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/CONT_AWD_EP1...,2017-09-30,2017-09-30
4,6800_6800_2005_0_EPD15041_0,CONT_AWD_2005_6800_EPD15041_6800,2005,0,0.0,6800.0,ENVIRONMENTAL PROTECTION AGENCY,EPD15041,0,41250.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.usaspending.gov/award/CONT_AWD_200...,2017-09-29,2022-01-05


In [4]:
# Data cleaning
data = data[~data['potential_total_value_of_award'].isnull()]  # Remove rows with missing potential total value
data['potential_total_value_of_award'] = data['potential_total_value_of_award'].astype(float)  # Convert to float

# Visualizations
fig = px.histogram(data, x='potential_total_value_of_award', title='Distribution of Potential Total Value of EPA Awards', nbins=50)
fig.update_xaxes(title='Potential Total Value (USD)')
fig.update_yaxes(title='Count')
fig.show()

fig = px.box(data, x='woman_owned_business', y='potential_total_value_of_award', title='Potential Total Value by Woman-Owned Business Status')
fig.update_xaxes(title='Woman-Owned Business')
fig.update_yaxes(title='Potential Total Value (USD)')
fig.show()

fig = px.box(data, x='minority_owned_business', y='potential_total_value_of_award', title='Potential Total Value by Minority-Owned Business Status')
fig.update_xaxes(title='Minority-Owned Business')
fig.update_yaxes(title='Potential Total Value (USD)')
fig.show()

In [5]:
# Funding by business type
business_type_summary = data.groupby('organizational_type')['potential_total_value_of_award'].sum().reset_index()

# Create the pie chart
fig = px.pie(business_type_summary, values='potential_total_value_of_award', names='organizational_type',
             title='Potential Total Value by Organizational Type',
             labels={'organizational_type': 'Organizational Type', 'potential_total_value_of_award': 'Potential Total Value (USD)'})

fig.show()


In [6]:
# Funding by product/service code (top 10)
product_service_code_summary = data.groupby('product_or_service_code_description')['potential_total_value_of_award'].sum().sort_values(ascending=False).head(10).reset_index()

# Create the pie chart
fig = px.pie(product_service_code_summary, values='potential_total_value_of_award', names='product_or_service_code_description',
             title='Potential Total Value by Product/Service Code (Top 10)',
             labels={'product_or_service_code_description': 'Product/Service Code Description', 'potential_total_value_of_award': 'Potential Total Value (USD)'})

fig.show()


In [7]:
# Calculate the mean award amount by NAICS description
mean_award_by_naics = data.groupby('naics_description')['potential_total_value_of_award'].mean().reset_index()

# Sort the data by mean award amount and select the top 10
mean_award_by_naics = mean_award_by_naics.sort_values(by='potential_total_value_of_award', ascending=False).head(10)

# Create the pie chart
fig = px.pie(mean_award_by_naics, values='potential_total_value_of_award', names='naics_description',
             title='Mean Award Amount by Company Type (Top 10)',
             labels={'naics_description': 'Company Type (NAICS Description)', 'potential_total_value_of_award': 'Mean Award Amount (USD)'})

fig.show()


In [8]:
# Women-owned businesses
women_owned = data[data['woman_owned_business'] == 't']
mean_award_women_owned = women_owned.groupby('naics_description')['potential_total_value_of_award'].mean().reset_index().sort_values(by='potential_total_value_of_award', ascending=False).head(10)

fig = px.pie(mean_award_women_owned, values='potential_total_value_of_award', names='naics_description', title='Mean Award Amount for Women-Owned Businesses (Top 10)')
fig.update_layout(title_x=0.5)
fig.show()

# Minority-owned businesses
minority_owned = data[data['minority_owned_business'] == 't']
mean_award_minority_owned = minority_owned.groupby('naics_description')['potential_total_value_of_award'].mean().reset_index().sort_values(by='potential_total_value_of_award', ascending=False).head(10)

fig = px.pie(mean_award_minority_owned, values='potential_total_value_of_award', names='naics_description', title='Mean Award Amount for Minority-Owned Businesses (Top 10)')
fig.update_layout(title_x=0.5)
fig.show()

# Small businesses
small_business = data[data['contracting_officers_determination_of_business_size_code'] == 'S']
mean_award_small_business = small_business.groupby('naics_description')['potential_total_value_of_award'].mean().reset_index().sort_values(by='potential_total_value_of_award', ascending=False).head(10)

fig = px.pie(mean_award_small_business, values='potential_total_value_of_award', names='naics_description', title='Mean Award Amount for Small Businesses (Top 10)')
fig.update_layout(title_x=0.5)
fig.show()